In [1]:
fp = f"./example/old_example/benchmarkF.asm"

In [2]:
from sample.cfg import proc_identify
from sample.isa import Instruction
from typing import Tuple
from sample.read_asm import AsmFileReader, StatementType

reader =AsmFileReader(fp)

statements = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

procs = proc_identify(statements)
for proc in procs:
    print(proc.name, proc.beg_addr.hex_str())
    if False and proc.name == 'main':
        for inst in proc.instruction:
            print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
        print()

_init 0000000000400c68
.plt 0000000000400c80
strlen@plt 0000000000400ca0
exit@plt 0000000000400cb0
__libc_start_main@plt 0000000000400cc0
execl@plt 0000000000400cd0
perror@plt 0000000000400ce0
listen@plt 0000000000400cf0
htonl@plt 0000000000400d00
bind@plt 0000000000400d10
sprintf@plt 0000000000400d20
pipe@plt 0000000000400d30
fork@plt 0000000000400d40
fclose@plt 0000000000400d50
atoi@plt 0000000000400d60
fopen@plt 0000000000400d70
ntohs@plt 0000000000400d80
setsockopt@plt 0000000000400d90
memset@plt 0000000000400da0
accept@plt 0000000000400db0
strcasecmp@plt 0000000000400dc0
htons@plt 0000000000400dd0
putenv@plt 0000000000400de0
close@plt 0000000000400df0
recv@plt 0000000000400e00
__gmon_start__@plt 0000000000400e10
stat@plt 0000000000400e20
write@plt 0000000000400e30
abort@plt 0000000000400e40
feof@plt 0000000000400e50
strcmp@plt 0000000000400e60
__ctype_b_loc@plt 0000000000400e70
send@plt 0000000000400e80
socket@plt 0000000000400e90
read@plt 0000000000400ea0
dup2@plt 0000000000400eb

In [3]:
from graphviz import Digraph
from sample.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./', format='svg')

'procedures.svg'

In [4]:
from sample.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./', format='svg')

'call_graph.svg'

In [5]:
from sample.cfg import TCfg

tcfg = TCfg(call_graph)
tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

for node in tcfg_nodes:
    print(node.name, node.base_proc.name, node.inst_range)

for edge in tcfg_edges:
    print(edge.src.name, edge.dst.name, edge.kind.name)

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./', format='svg')

n0 main (<sample.isa.Address object at 0x000002E3EFE25F90>, <sample.isa.Address object at 0x000002E3EFE271D0>)
n1 main (<sample.isa.Address object at 0x000002E3EFE27390>, <sample.isa.Address object at 0x000002E3EFE27B90>)
n2 main (<sample.isa.Address object at 0x000002E3EFE27D50>, <sample.isa.Address object at 0x000002E3EFE285D0>)
n3 main (<sample.isa.Address object at 0x000002E3EFE28790>, <sample.isa.Address object at 0x000002E3EFE28C90>)
n4 main (<sample.isa.Address object at 0x000002E3EFE28E90>, <sample.isa.Address object at 0x000002E3EFE29190>)
n5 main (<sample.isa.Address object at 0x000002E3EFE29350>, <sample.isa.Address object at 0x000002E3EFE29510>)
n6 main (<sample.isa.Address object at 0x000002E3EFE296D0>, <sample.isa.Address object at 0x000002E3EFE296D0>)
n7 main|4022c8#startup (<sample.isa.Address object at 0x000002E3EFE04DD0>, <sample.isa.Address object at 0x000002E3EFE05C90>)
n8 main|4022c8#startup (<sample.isa.Address object at 0x000002E3EFE05E50>, <sample.isa.Address ob

'tcfg.svg'